In [35]:
import numpy as np # linear algebra
import pandas as pd # data processing
import matplotlib as mplib
from matplotlib import pyplot as plt
import seaborn as sns
import pyro
import seaborn as sns

In [36]:
# for each country and year change the life expectancy to the difference between life expectancy of this and the next year
import time
data = pd.read_csv("Data/Data_processed.csv", sep=';')


for country in data['Country'].unique():
    d = data[data['Country'] == country]
    # sort by year
    d = d.sort_values(by='Year')
    # transform data to look at differences from year to year for all nummerical features (2 first columns are country and continent)
    d.iloc[:,5:] = d.iloc[:,5:].diff()
    # replace invividual nan values with 0
    data[data['Country'] == country] = d

data['Life expectancy difference'] = 0
for country in data['Country'].unique():
    d = data[data['Country'] == country]
    # sort by year
    d = d.sort_values(by='Year')
    # get difference in life expectancy
    d['Life expectancy difference'] = d['Life expectancy '].diff()


    # replace old values with new values
    data[data['Country'] == country] = d

# remove year 2000
# data = data[data['Year'] != 2000]
# remove other data that contains nan values
# data = data.dropna()

data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2936 entries, 0 to 2935
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Country                     2936 non-null   object 
 1   continent                   2936 non-null   object 
 2   Life expectancy             2936 non-null   float64
 3   Year                        2936 non-null   int64  
 4   Status                      2936 non-null   int64  
 5   infant deaths               2745 non-null   float64
 6   Alcohol                     2745 non-null   float64
 7   Hepatitis B                 2745 non-null   float64
 8   MeaslesPrMillion            2745 non-null   float64
 9   OverweightOfAdults%         2745 non-null   float64
 10  Polio                       2745 non-null   float64
 11  Total expenditure           2745 non-null   float64
 12  Diphtheria                  2745 non-null   float64
 13   HIV/AIDS                   2745 

In [37]:

# print(data.info())
# create a new column to store difference in life expectancy for 1, 2 and 3 years in the past
data['Life expectancy difference 1 year ago'] = 0
data['Life expectancy difference 2 years ago'] = 0
# get difference in life expectancy for each country and year
for country in data['Country'].unique():
    d = data[data['Country'] == country]
    # sort by year
    d = d.sort_values(by='Year')
    # get difference in life expectancy
    d['Life expectancy difference'] = d['Life expectancy difference'].shift(-1)
    d['Life expectancy difference 1 year ago'] = d['Life expectancy difference'].shift(1)
    d['Life expectancy difference 2 years ago'] = d['Life expectancy difference'].shift(2)

    # replace old values with new values
    data[data['Country'] == country] = d

# remove year 2001 and 2002
# data = data[data['Year'] != 2001]

# loop over each year and print how many nan values there are
# for year in data['Year'].unique():
#     print(year, data[data['Year'] == year].isna().sum().sum())

# print average over contries life expectancy difference for each year
data = data.dropna()
print(data.groupby('Year')['Life expectancy difference'].mean())



Year
2002    0.334970
2003    0.348231
2004    0.343238
2005    0.375969
2006    0.340868
2007    0.331219
2008    0.415480
2009    0.307339
2010    0.458009
2011    0.319320
2012    0.320209
2013    0.287139
2014    0.180493
Name: Life expectancy difference, dtype: float64


In [62]:
train_split = data[data['Year'] < 2009]
test_split = data[data['Year'] >= 2009]
test_split = test_split[test_split['Year'] < 2013]
val_split = data[data['Year'] >= 2013]

# make list of all columns that need to be standardized
cols = train_split.columns
cols = cols.drop(['Country', 'continent', 'Year', 'Status'])
# standardize all columns except country, continent, year and status
for col in cols:
    if col not in ['Country', 'continent', 'Year', 'Status']:
        mean = train_split[col].mean()
        std = train_split[col].std()
        train_split[col] = (train_split[col] - mean) / std
        test_split[col] = (test_split[col] - mean) / std
        val_split[col] = (val_split[col] - mean) / std

# make new data without target variable
cols = cols.drop(['Life expectancy difference', 'Life expectancy '])


# plot life expectancy for each country
# for i, country in enumerate(data['Country'].unique()):
#     d = data[data['Country'] == country]
#     plt.plot(d['Year'], d['Life expectancy difference'], label='Life expectancy ')
#     plt.plot(d['Year'], d['Life expectancy difference 1 year ago'], label='Life expectancy ')
#     plt.title(country)
#     plt.legend()
#     plt.show()
#     if i == 0:
#         break

C:\Users\jakob\AppData\Local\Temp\ipykernel_2496\1908549086.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_split[col] = (train_split[col] - mean) / std
C:\Users\jakob\AppData\Local\Temp\ipykernel_2496\1908549086.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_split[col] = (val_split[col] - mean) / std


In [63]:
# Fit linear model:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

# fit a regression model to predict the difference in life expectancy
reg = LinearRegression().fit(train_split[cols], train_split['Life expectancy difference'])
# for i, col in enumerate(cols):
#     print(col, reg.coef_[i])
y_pred_test = reg.predict(test_split[cols])
y_pred_val = reg.predict(val_split[cols])
mse_train = mean_squared_error(train_split['Life expectancy difference'], reg.predict(train_split[cols]))
mse_test = mean_squared_error(test_split['Life expectancy difference'], y_pred_test)
mse_val = mean_squared_error(val_split['Life expectancy difference'], y_pred_val)
print('Mean squared error: ', np.sqrt(mse_train), np.sqrt(mse_test), np.sqrt(mse_val))


# compare with baseline model that always predicts average of train data label
mse_baseline_train = mean_squared_error(train_split['Life expectancy difference'], np.zeros(len(train_split['Life expectancy difference'])))
mse_baseline_test = mean_squared_error(test_split['Life expectancy difference'], np.zeros(len(test_split['Life expectancy difference'])))
mse_baseline_val = mean_squared_error(val_split['Life expectancy difference'], np.zeros(len(val_split['Life expectancy difference'])))
print('Mean squared error baseline: ', np.sqrt(mse_baseline_train), np.sqrt(mse_baseline_test), np.sqrt(mse_baseline_val))



Mean squared error:  0.9296440303805034 1.6373452029262858 0.9290472070008566
Mean squared error baseline:  0.9996096037329261 1.6831051571269542 0.9596410303376235


In [8]:
# fit ridge regression model
from sklearn.linear_model import Ridge
reg = Ridge(alpha=100).fit(train_split[cols], train_split['Life expectancy difference'])
# predict the difference in life expectancy for the test set
y_pred = reg.predict(test_split[cols])

# calculate the mean squared error
mse = mean_squared_error(test_split['Life expectancy difference'], y_pred)
print('Mean squared error: ', mse)

for i, col in enumerate(cols):
    print(col, reg.coef_[i])


Mean squared error:  2.0829416288156533
infant deaths -0.18181991443626147
Alcohol 0.004108867725278199
Hepatitis B -0.007684747579936944
MeaslesPrMillion -0.01867009519964346
OverweightOfAdults% 0.03707408450567632
Polio 0.012227003020969443
Total expenditure 0.017452499626830473
Diphtheria  0.0024662277829400744
 HIV/AIDS -0.19141139116746791
GDP 0.029527973196946278
Population 0.04634269095032267
 thinness  1-19 years 0.02811851114002452
Schooling -0.020323200860828883
WaterFacility 0.05356292400665752
WomenInParlament -0.012765915821733363
Life expectancy difference 1 year ago -0.1132523980754056
Life expectancy difference 2 years ago 0.13104534141606583


In [42]:
# pyro linear dynamical system with inputs model (we assume that our other variables point to h, which then leads to life expectancy. 
# (Other variablies can be time dependent or not.) Here h would be like "life expectancy but without noise". Model is logical
# multi-variate state space model (we assume one hidden h eg. "Generel score" that leads to many observed variables. We don't necessarily
# assume that increasing the observed variable increases life expectancy. Model doesn't make as much sense as the one above.)
# different regimes? War vs not war? Pandemic vs not pandemic? Earthquake vs not earthquake? economic crisis vs not economic crisis? etc.
# probably better modelled with mixture model! Since so few "outlier" years where something like this happens. 
# output from h can flip a coin and then decide weight from maybe population or nothing!.

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import linear_model
import seaborn as sns
import time

import numpyro
import numpyro.distributions as dist
from numpyro import handlers
from numpyro.infer import MCMC, NUTS

import jax.numpy as jnp
from jax import random, vmap
from jax.scipy.special import logsumexp
from jax import lax

# fix random generator seed (for reproducibility of results)
np.random.seed(42)

# matplotlib style options
plt.style.use('ggplot')
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 10)


In [61]:
def f(carry, h_external_change):
  beta2, z_prev1, z_prev2 = carry
  z_t = beta2[0]*z_prev1 + beta2[1]*z_prev2 + h_external_change
  z_prev2 = z_prev1
  z_prev1 = z_t
  return (beta2, z_prev1, z_prev2), z_t


def model(obs_x, obs_y):
    time = obs_x.shape[0]
    countries = obs_x.shape[1]
    features_x = obs_x.shape[2]

    beta1 = numpyro.sample(name="beta1", fn=dist.Normal(loc=0., scale=1), sample_shape=(features_x,))
    xs = numpyro.sample(name="xs", fn=dist.Normal(loc=0., scale=1.), sample_shape=(time, countries, features_x, ), obs=obs_x)
    beta2 = numpyro.sample(name="beta2", fn=dist.Normal(loc=0., scale=1), sample_shape=(2,))
    tau = numpyro.sample(name="tau", fn=dist.HalfCauchy(scale=.1))
    sigma = numpyro.sample(name="sigma", fn=dist.HalfCauchy(scale=.1))
    z0 = numpyro.sample(name="z0", fn=dist.Normal(loc=0, scale=.1), sample_shape=(countries,))
    z1 = numpyro.sample(name="z1", fn=dist.Normal(loc=0, scale=.1), sample_shape=(countries,))
    h_external_change = numpyro.sample(name="h_change", fn=dist.Normal(loc=xs @ beta1, scale=tau))

    carry = (beta2, z0, z1)
    z_collection = [z0, z1]
    carry, zs_exp = lax.scan(f, carry, h_external_change, time)
    z_collection = jnp.concatenate((jnp.array(z_collection), zs_exp), axis=0)

    numpyro.sample(name="y_obs", fn=dist.Normal(loc=zs_exp, scale=sigma), obs=obs_y)
    return z_collection


(2,) (10,) (10,) (10,)


sample: 100%|██████████| 110/110 [00:08<00:00, 13.61it/s, 1 steps of size 5.00e-03. acc. prob=0.00]


Total time: 0.190m


In [158]:
# make list of all columns that need to be standardized
cols = train_split.columns
# cols = cols.drop(['Country', 'continent', 'Year', 'Status'])
# standardize all columns except country, continent, year and status
for col in cols:
    if col not in ['Country', 'continent', 'Year', 'Status']:
        mean = train_split[col].mean()
        std = train_split[col].std()
        train_split[col] = (train_split[col] - mean) / std
        test_split[col] = (test_split[col] - mean) / std

# make numpy array of standardized data, first dimension is time, second is countries, third is features
countries_train = train_split['Country'].unique()
years_train = train_split['Year'].unique()
countries_test = test_split['Country'].unique()
years_test = test_split['Year'].unique()
cols = cols.drop(['Life expectancy ', 'Country', 'Year', 'continent', 'Life expectancy difference 1 year ago', 'Life expectancy difference 2 years ago', 'Life expectancy difference'])
train_split_np_features = np.empty((len(years_train), len(countries_train), len(cols)))
train_split_np_target = np.empty((len(years_train), len(countries_train)))
test_split_np_features = np.empty((len(years_test), len(countries_test), len(cols)))
test_split_np_target = np.empty((len(years_test), len(countries_test)))

target = 'Life expectancy difference'
for i, country in enumerate(countries_train):
    for j, year in enumerate(years_train):
        # take nummeric data of country and put in numpy array
        train_split_np_features[j, i] = train_split.loc[(train_split['Country'] == country) & (train_split['Year'] == year)][cols].to_numpy()
        # take target of country and put in numpy array
        train_split_np_target[j, i] = train_split.loc[(train_split['Country'] == country) & (train_split['Year'] == year)][target].to_numpy()
for i, country in enumerate(countries_test):
    for j, year in enumerate(years_test):
        # take nummeric data of country and put in numpy array
        test_split_np_features[j, i] = test_split.loc[(test_split['Country'] == country) & (test_split['Year'] == year)][cols].to_numpy()
        # take target of country and put in numpy array
        test_split_np_target[j, i] = test_split.loc[(test_split['Country'] == country) & (test_split['Year'] == year)][target].to_numpy()

test_split_np_features = np.concatenate((train_split_np_features[-2:], test_split_np_features))
test_split_np_target = np.concatenate((train_split_np_target[-2:], test_split_np_target))


In [103]:
print(train_split_np_features.shape, test_split_np_features.shape, train_split_np_target.shape, test_split_np_target.shape)

t0 = time.time()

rng_key = random.PRNGKey(0)
rng_key, rng_key_ = random.split(rng_key)

nuts_kernel = NUTS(model=model, max_tree_depth=8, step_size=5e-3, adapt_step_size=False)
mcmc = MCMC(nuts_kernel, num_samples=4000, num_warmup=2000, num_chains=2)
mcmc.run(rng_key_, obs_x=train_split_np_features, obs_y=train_split_np_target)

t_fin = time.time()

print("Total time: {0:.3f}m".format((t_fin - t0)/60))

(7, 183, 16) (6, 183, 16) (7, 183) (6, 183)


C:\Users\jakob\AppData\Local\Temp\ipykernel_6728\2819762073.py:9: UserWarning: There are not enough devices to run parallel chains: expected 2 but got 1. Chains will be drawn sequentially. If you are running MCMC in CPU, consider using `numpyro.set_host_device_count(2)` at the beginning of your program. You can double-check how many devices are available in your system using `jax.local_device_count()`.
  mcmc = MCMC(nuts_kernel, num_samples=4000, num_warmup=2000, num_chains=2)
sample: 100%|██████████| 6000/6000 [10:52<00:00,  9.20it/s, 255 steps of size 5.00e-03. acc. prob=1.00]


Total time: 18.581m


In [130]:
hmc_samples = {k: v for k, v in mcmc.get_samples().items()}
for key, value in hmc_samples.items():
    print(key, value.shape)
# mcmc.print_summary()

beta1 (8000, 16)
beta2 (8000, 2)
h_change (8000, 7, 183)
sigma (8000,)
tau (8000,)
z0 (8000, 183)
z1 (8000, 183)


In [174]:
beta1 = mcmc.get_samples()['beta1'].mean(0)
beta2 = mcmc.get_samples()['beta2'].mean(0)
errors = []
errors_baseline = []
for i in range(2, test_split_np_features.shape[0]):
    prediction = beta2[0] * test_split_np_target[i-1] + beta2[1] * test_split_np_target[i-2] + test_split_np_features[i] @ beta1
    errors.append(np.mean((prediction - test_split_np_target[i])**2))
    errors_baseline.append(np.mean(test_split_np_target[i]**2))
print(np.mean(errors))
print(np.mean(errors_baseline))



2.0846033
2.1955322823340517
